In [ ]:
import numpy as np

def hermite_interpolate(x, fx, dfx):
    """
    Constructs the Hermite interpolating polynomial given:
        x   : 1D array-like with nodes x_0, x_1, ..., x_{n-1}  (length n)
        fx  : 1D array-like with function values f(x_i)
        dfx : 1D array-like with derivatives f'(x_i)

    Returns
    -------
    p : function
        A callable p(x_query) that evaluates the Hermite interpolating polynomial
        at x_query (scalar or array).
    
    Notes
    -----
    This function implements the standard Hermite interpolation approach by
    duplicating each node x_i to handle the function value and its first derivative:
    
    1) We create an array z of length 2n, where each x_i appears twice.
    2) We build a 2D array Q of size (2n x 2n) to store the divided differences:
       - Column 0 (Q[i,0]) is initialized with function values (and repeated, since
         each x_i is duplicated).
       - Q[2i+1, 1] is set to f'(x_i).
       - Q[2i,   1] is computed as the usual finite difference if i > 0.
    3) We fill out the rest of the columns in Q (from j=2 to j=2n-1) using the
       divided difference formula:
         Q[i, j] = (Q[i+1, j-1] - Q[i, j-1]) / (z[i+j] - z[i]).
    4) The resulting polynomial in Newton form can be written as:
         P(x) = Q[0,0] + Q[0,1]*(x - z_0) + Q[0,2]*(x - z_0)*(x - z_1) + ...
                 + Q[0, 2n-1]*Π_{m=0}^{2n-2} (x - z_m).
    """
    n = len(x)
    # Step 1: Build the array z by duplicating each x_i
    z = np.zeros(2*n)
    for i in range(n):
        z[2*i]   = x[i]
        z[2*i+1] = x[i]
        
    # Step 2: Initialize the divided-differences table Q
    Q = np.zeros((2*n, 2*n))
    
    # Step 3: Fill the first column of Q (function values f(x_i))
    for i in range(n):
        Q[2*i,   0] = fx[i]
        Q[2*i+1, 0] = fx[i]
    
    # According to Hermite interpolation:
    # - Q[2i+1, 1] = f'(x_i)
    # - If i > 0, Q[2i, 1] is a regular finite difference:
    #         (Q[2i, 0] - Q[2i-1, 0]) / (z[2i] - z[2i-1])
    for i in range(n):
        if 2*i+1 < 2*n:
            Q[2*i+1, 1] = dfx[i]
        if i > 0:
            Q[2*i, 1] = (Q[2*i, 0] - Q[2*i - 1, 0]) / (z[2*i] - z[2*i - 1])
    
    # Step 4: Fill out columns j = 2, 3, ..., 2n-1
    for j in range(2, 2*n):
        for i in range(2*n - j):
            Q[i, j] = (Q[i+1, j-1] - Q[i, j-1]) / (z[i+j] - z[i])
    
    def p(x_query):
        """
        Evaluate the Hermite interpolating polynomial (Newton form)
        at one or more points x_query.
        
        Parameters
        ----------
        x_query : float or 1D array-like
            The point(s) at which to evaluate the polynomial.
        
        Returns
        -------
        float or ndarray
            The interpolated value(s).
        """
        # Convert x_query to a numpy array to allow vectorized evaluations
        x_query = np.asarray(x_query, dtype=float)
        
        # Start with Q[0,0]
        total = np.full_like(x_query, Q[0, 0], dtype=float)
        
        # poly_factor accumulates (x - z_0), (x - z_0)(x - z_1), ...
        poly_factor = np.ones_like(x_query, dtype=float)
        
        # Add terms up to j = 2n-1
        for j in range(1, 2*n):
            poly_factor *= (x_query - z[j-1])
            total += Q[0, j] * poly_factor
        
        return total
    
    return p

In [ ]:
if __name__ == "__main__":
    import math
    import matplotlib.pyplot as plt

    # Let's do a simple example with 3 nodes: x=[0,1,2]
    x_nodes = [0, 1, 2]
    f_values = [math.sin(xi) for xi in x_nodes]
    df_values = [math.cos(xi) for xi in x_nodes]
    
    # Build the Hermite interpolating polynomial:
    hermite_poly = hermite_interpolate(x_nodes, f_values, df_values)
    
    # Evaluate on a grid and compare with sin(x)
    xs_eval = np.linspace(0, 2, 100)
    interp_vals = hermite_poly(xs_eval)
    true_vals   = np.sin(xs_eval)
    
    # Compute the maximum absolute error
    error = np.max(np.abs(interp_vals - true_vals))
    print(f"Maximum absolute error in [0, 2]: {error:.6e}")
    
    # Plot the results
    plt.figure(figsize=(7,4))
    plt.plot(xs_eval, true_vals, label="sin(x) (true)")
    plt.plot(xs_eval, interp_vals, '--', label="Hermite interpolation")
    plt.scatter(x_nodes, f_values, c='red', zorder=5, label="Nodes")
    plt.title("Hermite Interpolation Example")
    plt.legend()
    plt.show()